# Gibson Assembly Primer Designer for Binary Cloning

This notebook provides a complete tool for designing primers for **binary Gibson assembly**, where each step adds one insert to a vector.

## What You'll Get:
- **4 primers per assembly step** (2 for insert, 2 for vector)
- **Automatic Tm optimization** for efficient PCR
- **PCR simulation** to verify primer design
- **Assembly verification** using pydna
- **Multi-step planning** for sequential insertions

---

## Step 1: Install Required Packages

Run this cell first if you haven't installed these packages yet:

In [3]:
 #Uncomment and run if needed:
!pip install pydna biopython

## Step 2: Import Libraries

In [4]:
from typing import List, Dict, Tuple, Optional
from pydna.dseqrecord import Dseqrecord
from pydna.primer import Primer
from pydna.amplify import pcr
from pydna.assembly import Assembly
from Bio.SeqUtils import MeltingTemp as mt

print("✓ All libraries imported successfully!")

✓ All libraries imported successfully!


## Step 3: Helper Functions

These functions handle melting temperature calculations and circular DNA sequences:

In [5]:
def calculate_tm(seq: str) -> float:
    """Calculate melting temperature using nearest-neighbor method."""
    return mt.Tm_NN(seq, Na=50, dnac1=250, dnac2=250)


def get_circular_subseq(seq: str, start: int, length: int) -> str:
    """Get subsequence from circular DNA with wrap-around support."""
    n = len(seq)
    if length <= 0:
        return ''
    start = start % n
    if start + length <= n:
        return seq[start:start + length]
    else:
        return seq[start:] + seq[:(start + length) % n]


def find_optimal_anneal_length(template_seq: str, start_pos: int, 
                              direction: str, min_length: int, 
                              target_tm: float, tm_tolerance: float) -> Tuple[str, int, float]:
    """Find optimal annealing length to match target Tm."""
    max_length = min(40, len(template_seq))
    
    for length in range(min_length, max_length + 1):
        if direction == 'forward':
            anneal_seq = template_seq[start_pos:start_pos + length]
        else:
            anneal_seq = template_seq[max(0, start_pos - length):start_pos]
        
        if len(anneal_seq) < min_length:
            continue
            
        tm = calculate_tm(anneal_seq)
        
        if abs(tm - target_tm) <= tm_tolerance:
            return anneal_seq, length, tm
    
    # Use minimum length if no optimal found
    if direction == 'forward':
        anneal_seq = template_seq[start_pos:start_pos + min_length]
    else:
        anneal_seq = template_seq[max(0, start_pos - min_length):start_pos]
    
    tm = calculate_tm(anneal_seq)
    return anneal_seq, len(anneal_seq), tm

print("✓ Helper functions defined")

✓ Helper functions defined


## Step 3b: Primer Quality Analysis Functions

These functions check for common primer design issues like hairpins, dimers, and GC content:

In [6]:
from Bio.Seq import Seq
from Bio.SeqUtils import gc_fraction
import re


def check_gc_content(seq: str) -> Dict:
    """
    Check GC content of primer.
    Optimal: 40-60%
    """
    gc = gc_fraction(seq) * 100
    
    status = "✓ Good"
    if gc < 30 or gc > 70:
        status = "✗ Poor"
    elif gc < 40 or gc > 60:
        status = "⚠ Acceptable"
    
    return {
        'gc_percent': round(gc, 1),
        'status': status,
        'optimal': '40-60%'
    }


def check_gc_clamp(seq: str) -> Dict:
    """
    Check for GC clamp at 3' end.
    Good: 1-3 G/C in last 5 bases
    """
    last_5 = seq[-5:].upper()
    gc_count = last_5.count('G') + last_5.count('C')
    
    status = "✓ Good" if 1 <= gc_count <= 3 else "⚠ Weak"
    
    return {
        'gc_in_last_5': gc_count,
        'last_5_bases': last_5,
        'status': status,
        'optimal': '1-3 G/C bases'
    }


def check_runs(seq: str) -> Dict:
    """
    Check for runs of identical nucleotides.
    Warning: 4+ identical bases in a row
    """
    warnings = []
    
    for base in ['A', 'T', 'G', 'C']:
        pattern = base + '{4,}'
        matches = re.finditer(pattern, seq.upper())
        for match in matches:
            warnings.append(f"{len(match.group())} {base}'s at position {match.start()}")
    
    status = "✓ Good" if not warnings else "⚠ Runs detected"
    
    return {
        'runs': warnings if warnings else ['None'],
        'status': status
    }


def check_hairpin(seq: str, min_stem: int = 4, max_loop: int = 8) -> Dict:
    """
    Simple hairpin detection: checks if primer can fold back on itself.
    Looks for reverse complementary regions.
    
    This is a SIMPLIFIED check - for production use, consider:
    - primer3-py for accurate secondary structure prediction
    - NUPACK for comprehensive thermodynamic analysis
    """
    seq_upper = seq.upper()
    rev_comp = str(Seq(seq_upper).reverse_complement())
    
    warnings = []
    max_complementarity = 0
    
    # Check for complementarity between sequence and its reverse complement
    for i in range(len(seq) - min_stem):
        for j in range(len(seq) - min_stem):
            # Check complementarity
            matches = 0
            for k in range(min(min_stem, len(seq) - max(i, j))):
                if seq_upper[i + k] == rev_comp[j + k]:
                    matches += 1
            
            if matches >= min_stem:
                max_complementarity = max(max_complementarity, matches)
                if matches >= 4:
                    warnings.append(f"{matches} bp complementarity (positions {i}-{i+matches})")
    
    status = "✓ Good"
    if max_complementarity >= 6:
        status = "✗ Likely hairpin"
    elif max_complementarity >= 4:
        status = "⚠ Possible hairpin"
    
    return {
        'max_complementarity': max_complementarity,
        'warnings': warnings if warnings else ['None detected'],
        'status': status,
        'note': 'Simplified check - use primer3 for accurate analysis'
    }


def check_self_dimer(seq: str, min_complement: int = 4) -> Dict:
    """
    Check for self-complementarity (primer binding to itself).
    """
    seq_upper = seq.upper()
    rev_comp = str(Seq(seq_upper).reverse_complement())
    
    max_complementarity = 0
    warnings = []
    
    # Check alignment at different offsets
    for offset in range(-len(seq) + min_complement, len(seq) - min_complement + 1):
        matches = 0
        match_positions = []
        
        for i in range(len(seq)):
            j = i + offset
            if 0 <= j < len(seq):
                if seq_upper[i] == rev_comp[j]:
                    matches += 1
                    match_positions.append(i)
        
        if matches >= min_complement:
            max_complementarity = max(max_complementarity, matches)
            if matches >= 4:
                warnings.append(f"{matches} bp at offset {offset}")
    
    status = "✓ Good"
    if max_complementarity >= 6:
        status = "✗ Strong self-dimer"
    elif max_complementarity >= 4:
        status = "⚠ Possible self-dimer"
    
    return {
        'max_self_complementarity': max_complementarity,
        'warnings': warnings if warnings else ['None detected'],
        'status': status
    }


def check_primer_dimer(seq1: str, seq2: str, min_complement: int = 4) -> Dict:
    """
    Check for complementarity between two primers (hetero-dimer).
    """
    seq1_upper = seq1.upper()
    seq2_rev_comp = str(Seq(seq2.upper()).reverse_complement())
    
    max_complementarity = 0
    warnings = []
    
    # Check alignment at different offsets
    for offset in range(-len(seq1) + min_complement, len(seq2) - min_complement + 1):
        matches = 0
        
        for i in range(len(seq1)):
            j = i + offset
            if 0 <= j < len(seq2_rev_comp):
                if seq1_upper[i] == seq2_rev_comp[j]:
                    matches += 1
        
        if matches >= min_complement:
            max_complementarity = max(max_complementarity, matches)
            if matches >= 4:
                warnings.append(f"{matches} bp at offset {offset}")
    
    status = "✓ Good"
    if max_complementarity >= 6:
        status = "✗ Strong dimer"
    elif max_complementarity >= 4:
        status = "⚠ Possible dimer"
    
    return {
        'max_complementarity': max_complementarity,
        'warnings': warnings if warnings else ['None detected'],
        'status': status
    }


def analyze_primer_quality(seq: str, name: str = "Primer") -> Dict:
    """
    Comprehensive primer quality analysis.
    """
    return {
        'name': name,
        'sequence': seq,
        'length': len(seq),
        'tm': round(calculate_tm(seq), 1),
        'gc_content': check_gc_content(seq),
        'gc_clamp': check_gc_clamp(seq),
        'runs': check_runs(seq),
        'hairpin': check_hairpin(seq),
        'self_dimer': check_self_dimer(seq),
    }


def check_all_primer_pairs(primers: Dict) -> Dict:
    """
    Check all possible primer pair interactions.
    """
    primer_names = list(primers.keys())
    interactions = {}
    
    for i, name1 in enumerate(primer_names):
        for name2 in primer_names[i+1:]:
            pair_name = f"{name1}_vs_{name2}"
            interactions[pair_name] = check_primer_dimer(primers[name1], primers[name2])
    
    return interactions


def print_quality_report(primer_seq: str, primer_name: str):
    """
    Print a comprehensive quality report for a single primer.
    """
    analysis = analyze_primer_quality(primer_seq, primer_name)
    
    print(f"\n{'='*80}")
    print(f"PRIMER QUALITY REPORT: {primer_name}")
    print(f"{'='*80}")
    print(f"Sequence: {primer_seq}")
    print(f"Length:   {analysis['length']} bp")
    print(f"Tm:       {analysis['tm']}°C")
    
    print(f"\n--- GC Content ---")
    gc = analysis['gc_content']
    print(f"  {gc['status']}: {gc['gc_percent']}% (optimal: {gc['optimal']})")
    
    print(f"\n--- GC Clamp (3' stability) ---")
    clamp = analysis['gc_clamp']
    print(f"  {clamp['status']}: {clamp['gc_in_last_5']} G/C in last 5 bases ({clamp['last_5_bases']})")
    print(f"  Optimal: {clamp['optimal']}")
    
    print(f"\n--- Nucleotide Runs ---")
    runs = analysis['runs']
    print(f"  {runs['status']}")
    for run in runs['runs']:
        print(f"    {run}")
    
    print(f"\n--- Hairpin Formation ---")
    hairpin = analysis['hairpin']
    print(f"  {hairpin['status']}: Max {hairpin['max_complementarity']} bp self-complementarity")
    if hairpin['warnings'][0] != 'None detected':
        for warning in hairpin['warnings']:
            print(f"    {warning}")
    print(f"  Note: {hairpin['note']}")
    
    print(f"\n--- Self-Dimer ---")
    dimer = analysis['self_dimer']
    print(f"  {dimer['status']}: Max {dimer['max_self_complementarity']} bp self-complementarity")
    if dimer['warnings'][0] != 'None detected':
        for warning in dimer['warnings']:
            print(f"    {warning}")
    
    print(f"{'='*80}")


def print_dimer_analysis(primers: Dict):
    """
    Print analysis of primer-primer interactions.
    """
    print(f"\n{'='*80}")
    print(f"PRIMER PAIR INTERACTIONS (Hetero-Dimer Analysis)")
    print(f"{'='*80}")
    
    interactions = check_all_primer_pairs(primers)
    
    for pair_name, result in interactions.items():
        names = pair_name.split('_vs_')
        print(f"\n{names[0]} <-> {names[1]}:")
        print(f"  {result['status']}: {result['max_complementarity']} bp complementarity")
        if result['warnings'][0] != 'None detected':
            for warning in result['warnings']:
                print(f"    {warning}")
    
    print(f"\n{'='*80}")

print("✓ Primer quality analysis functions defined")

✓ Primer quality analysis functions defined


## Step 4: Main Primer Design Function

This is the core function that designs all 4 primers for a single assembly step:

In [7]:
def design_gibson_primers(vector: Dseqrecord, 
                         insert: Dseqrecord, 
                         insert_site: int,
                         homology_length: int = 40,
                         min_anneal_length: int = 20,
                         target_tm: float = 60.0,
                         tm_tolerance: float = 5.0) -> Dict:
    """
    Design 4 primers for binary Gibson assembly.
    
    Parameters:
    -----------
    vector : Dseqrecord
        Vector backbone (can be circular)
    insert : Dseqrecord
        Insert sequence to add
    insert_site : int
        Position in vector for insertion (0-based)
    homology_length : int
        Length of overlap for Gibson assembly (default: 40 bp)
    min_anneal_length : int
        Minimum annealing length (default: 20 bp)
    target_tm : float
        Target melting temperature (default: 60°C)
    tm_tolerance : float
        Acceptable Tm deviation (default: ±5°C)
    
    Returns:
    --------
    dict : Dictionary with primers, PCR products, and assembly result
    """
    
    # Ensure vector is circular
    if not vector.circular:
        print("⚠️  Vector set to circular")
        vector = vector.looped()
    
    # Get sequences as strings
    vec_seq = str(vector.seq)
    ins_seq = str(insert.seq)
    
    # Extract homology regions from vector (circular-aware)
    vec_left_homology = get_circular_subseq(vec_seq, insert_site - homology_length, homology_length)
    vec_right_homology = get_circular_subseq(vec_seq, insert_site, homology_length)
    
    # Find optimal annealing lengths for insert primers
    ins_fwd_anneal, ins_fwd_len, ins_fwd_tm = find_optimal_anneal_length(
        ins_seq, 0, 'forward', min_anneal_length, target_tm, tm_tolerance
    )
    ins_rev_anneal, ins_rev_len, ins_rev_tm = find_optimal_anneal_length(
        ins_seq, len(ins_seq), 'reverse', min_anneal_length, target_tm, tm_tolerance
    )
    
    # Design INSERT primers
    # Forward: vector left homology + insert forward annealing
    insert_fwd_seq = vec_left_homology + ins_fwd_anneal
    
    # Reverse: RC(vector right homology) + RC(insert reverse annealing)
    insert_rev_seq = str(Dseqrecord(vec_right_homology).reverse_complement().seq) + \
                    str(Dseqrecord(ins_rev_anneal).reverse_complement().seq)
    
    # Find optimal annealing lengths for vector primers
    vec_fwd_anneal, vec_fwd_len, vec_fwd_tm = find_optimal_anneal_length(
        vec_seq, insert_site, 'reverse', min_anneal_length, target_tm, tm_tolerance
    )
    vec_rev_anneal, vec_rev_len, vec_rev_tm = find_optimal_anneal_length(
        vec_seq, insert_site, 'forward', min_anneal_length, target_tm, tm_tolerance
    )
    
    # Design VECTOR primers
    # Forward: RC(insert left) + vector left annealing
    ins_left_homology = ins_seq[:homology_length]
    vector_fwd_seq = str(Dseqrecord(ins_left_homology).reverse_complement().seq) + vec_fwd_anneal
    
    # Reverse: insert right + RC(vector right annealing)
    ins_right_homology = ins_seq[-homology_length:]
    vector_rev_seq = ins_right_homology + str(Dseqrecord(vec_rev_anneal).reverse_complement().seq)
    
    # Create Primer objects
    insert_fwd_primer = Primer(insert_fwd_seq, name="Insert_Fwd")
    insert_rev_primer = Primer(insert_rev_seq, name="Insert_Rev")
    vector_fwd_primer = Primer(vector_fwd_seq, name="Vector_Fwd")
    vector_rev_primer = Primer(vector_rev_seq, name="Vector_Rev")
    
    # Simulate PCR products
    insert_pcr = pcr(insert_fwd_primer, insert_rev_primer, insert)
    vector_pcr = pcr(vector_fwd_primer, vector_rev_primer, vector)
    
    # Simulate Gibson assembly
    assembly = Assembly([insert_pcr, vector_pcr], limit=homology_length - 5)
    assembly_products = assembly.assemble_circular()
    
    # Compile results
    result = {
        'primers': {
            'insert_fwd': insert_fwd_seq,
            'insert_rev': insert_rev_seq,
            'vector_fwd': vector_fwd_seq,
            'vector_rev': vector_rev_seq,
        },
        'primer_objects': {
            'insert_fwd': insert_fwd_primer,
            'insert_rev': insert_rev_primer,
            'vector_fwd': vector_fwd_primer,
            'vector_rev': vector_rev_primer,
        },
        'pcr_products': {
            'insert': insert_pcr,
            'vector': vector_pcr,
        },
        'assembly_result': assembly_products[0] if assembly_products else None,
        'primer_details': {
            'insert_fwd': {
                'sequence': insert_fwd_seq,
                'length': len(insert_fwd_seq),
                'homology_tail': vec_left_homology,
                'homology_length': len(vec_left_homology),
                'anneal_seq': ins_fwd_anneal,
                'anneal_length': ins_fwd_len,
                'anneal_tm': round(ins_fwd_tm, 1),
                'full_tm': round(calculate_tm(insert_fwd_seq), 1),
            },
            'insert_rev': {
                'sequence': insert_rev_seq,
                'length': len(insert_rev_seq),
                'homology_tail': str(Dseqrecord(vec_right_homology).reverse_complement().seq),
                'homology_length': len(vec_right_homology),
                'anneal_seq': str(Dseqrecord(ins_rev_anneal).reverse_complement().seq),
                'anneal_length': ins_rev_len,
                'anneal_tm': round(ins_rev_tm, 1),
                'full_tm': round(calculate_tm(insert_rev_seq), 1),
            },
            'vector_fwd': {
                'sequence': vector_fwd_seq,
                'length': len(vector_fwd_seq),
                'homology_tail': str(Dseqrecord(ins_left_homology).reverse_complement().seq),
                'homology_length': len(ins_left_homology),
                'anneal_seq': vec_fwd_anneal,
                'anneal_length': vec_fwd_len,
                'anneal_tm': round(vec_fwd_tm, 1),
                'full_tm': round(calculate_tm(vector_fwd_seq), 1),
            },
            'vector_rev': {
                'sequence': vector_rev_seq,
                'length': len(vector_rev_seq),
                'homology_tail': ins_right_homology,
                'homology_length': len(ins_right_homology),
                'anneal_seq': str(Dseqrecord(vec_rev_anneal).reverse_complement().seq),
                'anneal_length': vec_rev_len,
                'anneal_tm': round(vec_rev_tm, 1),
                'full_tm': round(calculate_tm(vector_rev_seq), 1),
            },
        }
    }
    
    return result

print("✓ Main primer design function defined")

✓ Main primer design function defined


## Step 5: Multi-Step Assembly Planning

Plan sequential assembly steps where you add multiple inserts one by one:

In [8]:
def plan_multi_step_assembly(initial_vector: Dseqrecord, 
                            inserts: List[Dseqrecord], 
                            insert_site: int,
                            homology_length: int = 40,
                            target_tm: float = 60.0) -> List[Dict]:
    """
    Plan multi-step binary Gibson assembly.
    
    Parameters:
    -----------
    initial_vector : Dseqrecord
        Starting vector backbone
    inserts : List[Dseqrecord]
        List of inserts to add sequentially
    insert_site : int
        Position where each insert will be added
    homology_length : int
        Overlap length (default: 40 bp)
    target_tm : float
        Target Tm for primers (default: 60°C)
    
    Returns:
    --------
    List[dict] : List of primer designs for each step
    """
    
    steps = []
    current_vector = initial_vector
    
    for i, insert in enumerate(inserts, start=1):
        print(f"\n{'='*70}")
        print(f"Planning Step {i}: Adding insert '{insert.name}' ({len(insert)} bp)")
        print(f"{'='*70}")
        
        result = design_gibson_primers(
            vector=current_vector,
            insert=insert,
            insert_site=insert_site,
            homology_length=homology_length,
            target_tm=target_tm
        )
        
        result['step'] = i
        result['insert'] = insert
        result['input_vector'] = current_vector
        
        steps.append(result)
        
        # Update vector for next step
        if result['assembly_result']:
            current_vector = result['assembly_result']
            print(f"✓ Assembly successful! New vector: {len(current_vector)} bp")
        else:
            print(f"✗ Warning: Assembly prediction failed for step {i}")
            break
    
    return steps

print("✓ Multi-step planning function defined")

✓ Multi-step planning function defined


## Step 6: Results Display Function

Pretty-print the primer design results:

In [9]:
def print_primer_summary(result: Dict, show_sequences: bool = True):
    """
    Print a formatted summary of primer design.
    
    Parameters:
    -----------
    result : dict
        Result from design_gibson_primers()
    show_sequences : bool
        Whether to show full primer sequences (default: True)
    """
    
    print("\n" + "="*80)
    print("GIBSON ASSEMBLY PRIMER DESIGN SUMMARY")
    print("="*80)
    
    for primer_name in ['insert_fwd', 'insert_rev', 'vector_fwd', 'vector_rev']:
        details = result['primer_details'][primer_name]
        
        print(f"\n{primer_name.upper().replace('_', ' ')}:")
        
        if show_sequences:
            print(f"  Sequence (5'→3'): {details['sequence']}")
        
        print(f"  Total Length: {details['length']} bp")
        print(f"  Homology Tail: {details['homology_length']} bp (for Gibson overlap)")
        print(f"  Annealing Region: {details['anneal_length']} bp (Tm: {details['anneal_tm']}°C)")
        print(f"  Full Primer Tm: {details['full_tm']}°C")
    
    print("\n" + "="*80)
    print("PCR PRODUCTS:")
    print("="*80)
    print(f"Insert PCR product: {len(result['pcr_products']['insert'])} bp")
    print(f"Vector PCR product: {len(result['pcr_products']['vector'])} bp")
    
    if result['assembly_result']:
        print(f"\n✓ Assembly Product: {len(result['assembly_result'])} bp (circular)")
    else:
        print(f"\n✗ Assembly failed - check homology regions")
    
    print("="*80)


def get_primers_for_ordering(result: Dict) -> None:
    """
    Print primers in a format ready for ordering.
    """
    print("\n" + "="*80)
    print("PRIMERS FOR ORDERING (5' → 3')")
    print("="*80)
    
    for name, seq in result['primers'].items():
        print(f"\n{name.upper()}:")
        print(f"  {seq}")
        print(f"  Length: {len(seq)} bp")
    
    print("\n" + "="*80)

print("✓ Display functions defined")

✓ Display functions defined


---

# 🎯 USAGE EXAMPLES

Below are examples showing how to use the primer designer.

### 🔧 Troubleshooting Common Errors

**Error: "ValueError: PCR not specific!"**
- **Cause:** Primers bind to multiple locations in your sequence
- **Common reason:** Using highly repetitive test sequences
- **Solution:** Use more diverse sequences, or adjust `insert_site` to a more unique region
- **In real use:** This error means you need to redesign - your primers aren't specific enough!

**Error: "NameError: name 'design_gibson_primers' is not defined"**
- **Cause:** Functions not loaded
- **Solution:** Run cells 3-11 first (the function definition cells)

**Error: "ImportError: No module named 'pydna'"**
- **Cause:** Package not installed
- **Solution:** Run cell 3: `!pip install pydna biopython`

---

## Example 1: Single Insert - Basic Usage

Design primers to insert one gene into a plasmid:

### ⚠️ Important Note About Test Sequences

The example uses a **random non-repetitive sequence** for the vector. Why?

**If you use repetitive sequences** (like `"ATGC" * 200`), pydna will correctly detect that primers bind to multiple locations and report:
```
ValueError: PCR not specific!
```

This is a **good thing** - pydna is protecting you from bad primer design! In real lab work:
- Your actual plasmid won't have perfect repeats
- If you get this error with real sequences, your primers genuinely have specificity issues
- Use more unique sequences or adjust the insert_site position

---

In [17]:
# Create a circular plasmid vector (more realistic, non-repetitive sequence)
# Using a diverse sequence to avoid multiple primer binding sites
import random
random.seed(42)  # For reproducibility
vector_seq = ''.join(random.choices('ATGC', k=800))
vector = Dseqrecord(vector_seq, circular=True, name="pVector")

# Create an insert (your gene of interest)
insert_seq = "GGGAAAATTTCCCGGGTTTAAACCCGGGAAAATTTCCCGGGTTTAAA"
insert = Dseqrecord(insert_seq, name="GeneOfInterest")

print(f"Vector: {len(vector)} bp (circular)")
print(f"Insert: {len(insert)} bp")
print(f"Insert will be placed at position 100")

Vector: 800 bp (circular)
Insert: 47 bp
Insert will be placed at position 100


In [18]:
# Design the primers
result = design_gibson_primers(
    vector=vector,
    insert=insert,
    insert_site=100,       # Position in vector
    homology_length=40,    # 40 bp overlap
    target_tm=60.0         # Target 60°C Tm
)

# Display results
print_primer_summary(result)


GIBSON ASSEMBLY PRIMER DESIGN SUMMARY

INSERT FWD:
  Sequence (5'→3'): CGGGCCAATTACCTGTCTTAGTGCTACGAAAGCTATCGCCGGGAAAATTTCCCGGGTTTAAAC
  Total Length: 63 bp
  Homology Tail: 40 bp (for Gibson overlap)
  Annealing Region: 23 bp (Tm: 55.6°C)
  Full Primer Tm: 73.7°C

INSERT REV:
  Sequence (5'→3'): TGGTATGGTAGGGTATCGCGTCCAGGTCAGGAATCACCCTTTTAAACCCGGGAAATTTTCCCG
  Total Length: 63 bp
  Homology Tail: 40 bp (for Gibson overlap)
  Annealing Region: 23 bp (Tm: 56.9°C)
  Full Primer Tm: 74.3°C

VECTOR FWD:
  Sequence (5'→3'): CCGGGAAATTTTCCCGGGTTTAAACCCGGGAAATTTTCCCGTGCTACGAAAGCTATCGCC
  Total Length: 60 bp
  Homology Tail: 40 bp (for Gibson overlap)
  Annealing Region: 20 bp (Tm: 56.6°C)
  Full Primer Tm: 74.7°C

VECTOR REV:
  Sequence (5'→3'): TTTCCCGGGTTTAAACCCGGGAAAATTTCCCGGGTTTAAATCCAGGTCAGGAATCACCCT
  Total Length: 60 bp
  Homology Tail: 40 bp (for Gibson overlap)
  Annealing Region: 20 bp (Tm: 57.1°C)
  Full Primer Tm: 73.8°C

PCR PRODUCTS:
Insert PCR product: 127 bp
Vector PCR produc

In [ ]:
# Get primers in ordering format
get_primers_for_ordering(result)

## Example 2: Multi-Step Sequential Assembly

Add multiple inserts one by one:

In [ ]:
# Create multiple inserts
insert1 = Dseqrecord("GGGAAAATTTCCCGGGTTTAAACCCGGG", name="Insert1")
insert2 = Dseqrecord("TTTGGGCCCAAAATTTGGGCCCAAATTT", name="Insert2")
insert3 = Dseqrecord("AAACCCTTTGGGAAATTTCCCGGGTAAA", name="Insert3")

inserts = [insert1, insert2, insert3]

print(f"Planning assembly for {len(inserts)} inserts")

In [ ]:
# Plan all assembly steps
steps = plan_multi_step_assembly(
    initial_vector=vector,
    inserts=inserts,
    insert_site=100,
    homology_length=40
)

In [ ]:
# Display primers for each step
for step in steps:
    print(f"\n\n{'#'*80}")
    print(f"STEP {step['step']} - Insert: {step['insert'].name}")
    print(f"{'#'*80}")
    print_primer_summary(step, show_sequences=False)  # Hide sequences for brevity

In [ ]:
# Get primers for a specific step (e.g., Step 1)
step_number = 1
print(f"\nPrimers for Step {step_number}:")
get_primers_for_ordering(steps[step_number - 1])

## Example 3: Load Sequences from Files

Work with real GenBank or FASTA files:

## Example 3A: Working with FASTA Format

You can input sequences in FASTA format in several ways:
1. **From FASTA files** on your computer
2. **Direct FASTA strings** in the notebook
3. **Download from NCBI** using accession numbers

Let's try all three methods!

### Method 1: FASTA String Directly in Notebook

You can paste FASTA format sequences directly:

In [ ]:
from Bio import SeqIO
from io import StringIO

# Example 1: Real pUC19 vector (common cloning vector, 2686 bp)
puc19_fasta = """>pUC19
TCGCGCGTTTCGGTGATGACGGTGAAAACCTCTGACACATGCAGCTCCCGGAGACGGTCACAGCTTGTC
TGTAAGCGGATGCCGGGAGCAGACAAGCCCGTCAGGGCGCGTCAGCGGGTGTTGGCGGGTGTCGGGGCT
GGCTTAACTATGCGGCATCAGAGCAGATTGTACTGAGAGTGCACCATATGCGGTGTGAAATACCGCACA
GATGCGTAAGGAGAAAATACCGCATCAGGCGCCATTCGCCATTCAGGCTGCGCAACTGTTGGGAAGGGC
GATCGGTGCGGGCCTCTTCGCTATTACGCCAGCTGGCGAAAGGGGGATGTGCTGCAAGGCGATTAAGTT
GGGTAACGCCAGGGTTTTCCCAGTCACGACGTTGTAAAACGACGGCCAGTGAATTCGAGCTCGGTACCC
GGGGATCCTCTAGAGTCGACCTGCAGGCATGCAAGCTTGGCGTAATCATGGTCATAGCTGTTTCCTGTG
TGAAATTGTTATCCGCTCACAATTCCACACAACATACGAGCCGGAAGCATAAAGTGTAAAGCCTGGGGT
GCCTAATGAGTGAGCTAACTCACATTAATTGCGTTGCGCTCACTGCCCGCTTTCCAGTCGGGAAACCTG
TCGTGCCAGCTGCATTAATGAATCGGCCAACGCGCGGGGAGAGGCGGTTTGCGTATTGGGCGCTCTTCC
GCTTCCTCGCTCACTGACTCGCTGCGCTCGGTCGTTCGGCTGCGGCGAGCGGTATCAGCTCACTCAAAG
GCGGTAATACGGTTATCCACAGAATCAGGGGATAACGCAGGAAAGAACATGTGAGCAAAAGGCCAGCAA
AAGGCCAGGAACCGTAAAAAGGCCGCGTTGCTGGCGTTTTTCCATAGGCTCCGCCCCCCTGACGAGCAT
CACAAAAATCGACGCTCAAGTCAGAGGTGGCGAAACCCGACAGGACTATAAAGATACCAGGCGTTTCCC
CCTGGAAGCTCCCTCGTGCGCTCTCCTGTTCCGACCCTGCCGCTTACCGGATACCTGTCCGCCTTTCTC
CCTTCGGGAAGCGTGGCGCTTTCTCATAGCTCACGCTGTAGGTATCTCAGTTCGGTGTAGGTCGTTCGC
TCCAAGCTGGGCTGTGTGCACGAACCCCCCGTTCAGCCCGACCGCTGCGCCTTATCCGGTAACTATCGT
CTTGAGTCCAACCCGGTAAGACACGACTTATCGCCACTGGCAGCAGCCACTGGTAACAGGATTAGCAGA
GCGAGGTATGTAGGCGGTGCTACAGAGTTCTTGAAGTGGTGGCCTAACTACGGCTACACTAGAAGGACA
GTATTTGGTATCTGCGCTCTGCTGAAGCCAGTTACCTTCGGAAAAAGAGTTGGTAGCTCTTGATCCGGC
AAACAAACCACCGCTGGTAGCGGTGGTTTTTTTGTTTGCAAGCAGCAGATTACGCGCAGAAAAAAAGGA
TCTCAAGAAGATCCTTTGATCTTTTCTACGGGGTCTGACGCTCAGTGGAACGAAAACTCACGTTAAGGG
ATTTTGGTCATGAGATTATCAAAAAGGATCTTCACCTAGATCCTTTTAAATTAAAAATGAAGTTTTAAA
TCAATCTAAAGTATATATGAGTAAACTTGGTCTGACAGTTACCAATGCTTAATCAGTGAGGCACCTATC
TCAGCGATCTGTCTATTTCGTTCATCCATAGTTGCCTGACTCCCCGTCGTGTAGATAACTACGATACGG
GAGGGCTTACCATCTGGCCCCAGTGCTGCAATGATACCGCGAGACCCACGCTCACCGGCTCCAGATTTA
TCAGCAATAAACCAGCCAGCCGGAAGGGCCGAGCGCAGAAGTGGTCCTGCAACTTTATCCGCCTCCATC
CAGTCTATTAATTGTTGCCGGGAAGCTAGAGTAAGTAGTTCGCCAGTTAATAGTTTGCGCAACGTTGTT
GCCATTGCTACAGGCATCGTGGTGTCACGCTCGTCGTTTGGTATGGCTTCATTCAGCTCCGGTTCCCAA
CGATCAAGGCGAGTTACATGATCCCCCATGTTGTGCAAAAAAGCGGTTAGCTCCTTCGGTCCTCCGATC
GTTGTCAGAAGTAAGTTGGCCGCAGTGTTATCACTCATGGTTATGGCAGCACTGCATAATTCTCTTACT
GTCATGCCATCCGTAAGATGCTTTTCTGTGACTGGTGAGTACTCAACCAAGTCATTCTGAGAATAGTGT
ATGCGGCGACCGAGTTGCTCTTGCCCGGCGTCAATACGGGATAATACCGCGCCACATAGCAGAACTTTA
AAAGTGCTCATCATTGGAAAACGTTCTTCGGGGCGAAAACTCTCAAGGATCTTACCGCTGTTGAGATCC
AGTTCGATGTAACCCACTCGTGCACCCAACTGATCTTCAGCATCTTTTACTTTCACCAGCGTTTCTGGG
TGAGCAAAAACAGGAAGGCAAAATGCCGCAAAAAAGGGAATAAGGGCGACACGGAAATGTTGAATACTC
ATACTCTTCCTTTTTCAATATTATTGAAGCATTTATCAGGGTTATTGTCTCATGAGCGGATACATATTT
GAATGTATTTAGAAAAATAAACAAATAGGGGTTCCGCGCACATTTCCCCGAAAAGTGCCACCTGACGTC
TAAGAAACCATTATTATCATGACATTAACCTATAAAAATAGGCGTATCACGAGGCCCTTTCGTC
"""

# Parse using BioPython and convert to pydna
seq_record = SeqIO.read(StringIO(puc19_fasta), "fasta")
vector = Dseqrecord(seq_record)
vector.circular = True  # pUC19 is circular
vector.name = "pUC19"

print(f"Loaded vector: {vector.name}")
print(f"Length: {len(vector)} bp")
print(f"Circular: {vector.circular}")
print(f"First 60 bp: {str(vector.seq)[:60]}...")

No sequence found in data:
(<_io.StringIO object at 0x00000234FED21540>)


ValueError: not enough values to unpack (expected 1, got 0)

In [ ]:
# Example 2: Real ORF insert - GFP (Green Fluorescent Protein, 720 bp)
gfp_fasta = """>GFP_ORF Green Fluorescent Protein coding sequence
ATGAGTAAAGGAGAAGAACTTTTCACTGGAGTTGTCCCAATTCTTGTTGAATTAGATGGTGATGTTAAT
GGGCACAAATTTTCTGTCAGTGGAGAGGGTGAAGGTGATGCAACATACGGAAAACTTACCCTTAAATTT
ATTTGCACTACTGGAAAACTACCTGTTCCATGGCCAACACTTGTCACTACTTTCTCTTATGGTGTTCAA
TGCTTTTCCCGTTATCCGGATCATATGAAACGGCATGACTTTTTCAAGAGTGCCATGCCCGAAGGTTAT
GTACAGGAACGCACTATATCTTTCAAAGATGACGGGAACTACAAGACGCGTGCTGAAGTCAAGTTTGAA
GGTGATACCCTTGTTAATCGTATCGAGTTAAAAGGTATTGATTTTAAAGAAGATGGAAACATTCTCGGA
CACAAACTCGAGTACAACTATAACTCACACAATGTATACATCACGGCAGACAAACAAAAGAATGGAATC
AAAGCTAACTTCAAAATTCGCCACAACATTGAAGATGGAAGCGTTCAACTAGCAGACCATTATCAACAA
AATACTCCAATTGGCGATGGCCCTGTCCTTTTACCAGACAACCATTACCTGTCGACACAATCTGCCCTTT
CGAAAGATCCCAACGAAAAGCGTGACCACATGGTCCTTCTTGAGTTTGTAACTGCTGCTGGGATTACAC
ATGGCATGGATGAGCTCTACAAATAA
"""

insert = read(StringIO(gfp_fasta))[0]
insert.name = "GFP"

print(f"\\nLoaded insert: {insert.name}")
print(f"Length: {len(insert)} bp")
print(f"First 60 bp: {str(insert.seq)[:60]}...")
print(f"Last 60 bp: ...{str(insert.seq)[-60:]}")

In [ ]:
# Now design primers to insert GFP into pUC19 at the EcoRI site (position 396)
# This is a common cloning strategy!

result_real = design_gibson_primers(
    vector=vector,
    insert=insert,
    insert_site=396,        # EcoRI site in pUC19
    homology_length=40,
    target_tm=60.0
)

print("\\n" + "="*80)
print("DESIGNING PRIMERS TO INSERT GFP INTO pUC19")
print("="*80)
print_primer_summary(result_real)

In [ ]:
# Get the primers ready for ordering
get_primers_for_ordering(result_real)

### Method 2: Load from FASTA Files

If you have FASTA files saved on your computer:

In [ ]:
# First, let's create example FASTA files
# (In real use, you'd already have these files)

# Create a FASTA file for the vector
with open('pUC19_vector.fasta', 'w') as f:
    f.write(puc19_fasta)

# Create a FASTA file for the insert
with open('GFP_insert.fasta', 'w') as f:
    f.write(gfp_fasta)

print("✓ Created example FASTA files:")
print("  - pUC19_vector.fasta")
print("  - GFP_insert.fasta")

In [ ]:
# Now load from the FASTA files
vector_from_file = read("pUC19_vector.fasta")[0]
vector_from_file.circular = True
vector_from_file.name = "pUC19"

insert_from_file = read("GFP_insert.fasta")[0]
insert_from_file.name = "GFP"

print(f"✓ Loaded from files:")
print(f"  Vector: {vector_from_file.name} ({len(vector_from_file)} bp)")
print(f"  Insert: {insert_from_file.name} ({len(insert_from_file)} bp)")

# Design primers
result_from_files = design_gibson_primers(
    vector=vector_from_file,
    insert=insert_from_file,
    insert_site=396,
    homology_length=40
)

print("\\n✓ Primers designed successfully from FASTA files!")

### Method 3: Download from NCBI using Accession Numbers

You can automatically download sequences from NCBI GenBank using their accession numbers. This is convenient when you know the database ID.

In [ ]:
from Bio import Entrez, SeqIO

# IMPORTANT: Replace with your email (NCBI requires this)
Entrez.email = "your.email@example.com"

# Download pUC19 sequence from NCBI (Accession: M77789)
print("Downloading pUC19 from NCBI (M77789)...")
handle = Entrez.efetch(db="nucleotide", id="M77789", rettype="fasta", retmode="text")
puc19_ncbi = SeqIO.read(handle, "fasta")
handle.close()

# Convert to pydna Dseqrecord
from pydna.dseqrecord import Dseqrecord
vector_ncbi = Dseqrecord(puc19_ncbi)
vector_ncbi.circular = True
vector_ncbi.name = "pUC19"

print(f"✓ Downloaded: {vector_ncbi.name} ({len(vector_ncbi)} bp)")
print(f"  Description: {puc19_ncbi.description}")

# You can use this vector just like the others:
# result_from_ncbi = design_gibson_primers(vector_ncbi, insert_gfp, insert_site=396)

### Bonus: Working with GenBank Files

If you have GenBank (.gb) files with annotations, you can load them too:

In [ ]:
# Example: If you have a GenBank file
# from pydna.readers import read as read_gb
# vector_gb = read_gb("plasmid.gb")[0]
# vector_gb.circular = True  # if circular

# GenBank files preserve annotations (genes, promoters, etc.)
# which can be useful for tracking features after assembly

print("💡 GenBank format preserves feature annotations (genes, promoters, etc.)")
print("   Load with: read('file.gb')[0]")

In [ ]:
# Example code for loading from files
# Uncomment and modify paths as needed:

# from pydna.readers import read
# 
# # Load vector from GenBank file
# vector = read("your_plasmid.gb")[0]
# 
# # Load insert from FASTA file
# insert = read("your_gene.fasta")[0]
# 
# # Design primers
# result = design_gibson_primers(vector, insert, insert_site=500)
# print_primer_summary(result)

print("See commented code above for file loading example")

## Example 4: Customize Homology Regions

Control where the homology regions fall by adjusting parameters:

In [ ]:
# Try different homology lengths
print("Testing different homology lengths:\n")

for homology in [20, 30, 40, 50]:
    result = design_gibson_primers(
        vector=vector,
        insert=insert,
        insert_site=100,
        homology_length=homology
    )
    
    print(f"\nHomology length: {homology} bp")
    print(f"  Insert_Fwd length: {result['primer_details']['insert_fwd']['length']} bp")
    print(f"  Insert_Rev length: {result['primer_details']['insert_rev']['length']} bp")

## Example 5: Examine Detailed Primer Information

In [ ]:
# Get detailed information about a specific primer
primer_name = 'insert_fwd'
details = result['primer_details'][primer_name]

print(f"Detailed Analysis of {primer_name.upper()}:")
print("="*80)
print(f"\nFull Sequence:")
print(f"  {details['sequence']}")
print(f"\nStructure:")
print(f"  5' Homology Tail: {details['homology_tail'][:30]}... ({details['homology_length']} bp)")
print(f"  3' Annealing:     {details['anneal_seq']} ({details['anneal_length']} bp)")
print(f"\nThermal Properties:")
print(f"  Annealing Tm: {details['anneal_tm']}°C")
print(f"  Full Tm:      {details['full_tm']}°C")
print(f"\nTotal Length: {details['length']} bp")

## Example 6: Verify Assembly Success

In [ ]:
# Check the assembly result
if result['assembly_result']:
    assembly = result['assembly_result']
    
    print("Assembly Verification:")
    print("="*80)
    print(f"✓ Assembly SUCCESSFUL!")
    print(f"\nInput sizes:")
    print(f"  Original vector:     {len(vector)} bp")
    print(f"  Insert:              {len(insert)} bp")
    print(f"\nPCR products:")
    print(f"  Insert PCR product:  {len(result['pcr_products']['insert'])} bp")
    print(f"  Vector PCR product:  {len(result['pcr_products']['vector'])} bp")
    print(f"\nFinal assembly:")
    print(f"  Final plasmid:       {len(assembly)} bp")
    print(f"  Circular:            {assembly.circular}")
    print(f"  Name:                {assembly.name}")
else:
    print("✗ Assembly failed! Check your parameters.")

---

## 📝 Quick Reference

### Main Function Parameters:

```python
design_gibson_primers(
    vector=your_vector,        # Dseqrecord (circular plasmid)
    insert=your_insert,        # Dseqrecord (gene to insert)
    insert_site=100,           # Position in vector (0-based)
    homology_length=40,        # Overlap length (20-80 bp recommended)
    min_anneal_length=20,      # Minimum annealing (≥18 bp)
    target_tm=60.0,            # Target melting temp (°C)
    tm_tolerance=5.0           # Tm tolerance (±°C)
)
```

### Output Structure:

```python
result = {
    'primers': dict,           # Primer sequences as strings
    'primer_objects': dict,    # pydna Primer objects
    'pcr_products': dict,      # Simulated PCR products
    'assembly_result': obj,    # Assembled plasmid
    'primer_details': dict     # Detailed info (Tm, lengths, etc.)
}
```

### Tips:

1. **Homology length**: 40 bp is optimal for Gibson assembly
2. **Insert site**: Choose a unique position in your vector
3. **Tm values**: Primers designed for similar Tm (~60°C)
4. **Circular plasmids**: Always set `circular=True` for vectors
5. **Multi-step**: Each step updates the vector automatically

---

## 🚀 YOUR CUSTOM PRIMER DESIGN

Use the cell below to design primers for your own sequences:

## 🔬 Example 7: Comprehensive Primer Quality Analysis

Check your primers for common issues like hairpins, dimers, GC content, etc.:

In [ ]:
# Design primers first (reusing example from earlier)
# Note: Using the vector and insert from Example 1 (already defined above)
# If running this independently, uncomment the lines below:

# import random
# random.seed(42)
# vector = Dseqrecord(''.join(random.choices('ATGC', k=800)), circular=True, name="pVector")
# insert = Dseqrecord("GGGAAAATTTCCCGGGTTTAAACCCGGGAAAATTTCCCGGGTTTAAA", name="Insert")

result = design_gibson_primers(vector, insert, insert_site=100, homology_length=40)

print("Primers designed! Now analyzing quality...\n")

In [ ]:
# Analyze quality of each primer individually
for primer_name, primer_seq in result['primers'].items():
    print_quality_report(primer_seq, primer_name.upper())

In [ ]:
# Check for primer-primer interactions (hetero-dimers)
print_dimer_analysis(result['primers'])

### Understanding the Quality Checks:

#### **What This Tool Checks:**

1. **Tm (Melting Temperature)** 
   - Method: Nearest-Neighbor thermodynamics (SantaLucia 1998)
   - Considers: Base stacking, salt concentration (50mM Na+), primer concentration (250nM)
   - More accurate than simple GC% methods

2. **GC Content**
   - ✓ Optimal: 40-60%
   - ⚠ Acceptable: 30-70%
   - ✗ Poor: <30% or >70%

3. **GC Clamp** 
   - Checks 3' end stability
   - ✓ Good: 1-3 G/C bases in last 5 positions
   - Helps primer bind firmly

4. **Nucleotide Runs**
   - Warns about poly-A, poly-T, poly-G, poly-C stretches
   - 4+ identical bases can cause mispriming

5. **Hairpin Formation** (simplified check)
   - Detects self-complementarity (primer folding)
   - ✓ Good: <4 bp complementarity
   - ⚠ Warning: 4-5 bp
   - ✗ Problem: ≥6 bp

6. **Self-Dimer**
   - Primer binding to itself in same direction
   - Same thresholds as hairpin

7. **Primer-Primer Dimers** (hetero-dimers)
   - Two different primers binding to each other
   - Can compete with target amplification

#### **Limitations of This Implementation:**

⚠️ **These are SIMPLIFIED checks** - for production/critical work, use:
- **Primer3** (via primer3-py) - gold standard for primer design
- **NUPACK** - accurate thermodynamic calculations
- **IDT OligoAnalyzer** - comprehensive commercial tool

This tool gives you a quick assessment, but doesn't:
- Calculate exact ΔG for secondary structures
- Model complete folding pathways
- Account for all PCR conditions
- Check for off-target binding in genome

#### **Interpreting Results:**

- **✓ Good**: No issues detected
- **⚠ Acceptable/Warning**: Minor concern, usually OK but monitor
- **✗ Problem**: Likely to cause issues, consider redesigning

For Gibson assembly primers:
- Some dimers are expected (they have long homology tails!)
- Focus on the 3' annealing region quality
- Hairpins in homology tail are less critical than in annealing region

In [ ]:
# YOUR CUSTOM DESIGN HERE
# Replace these sequences with your own:

my_vector_seq = "ATGCGTACGT" * 50  # Replace with your vector sequence
my_vector = Dseqrecord(my_vector_seq, circular=True, name="MyVector")

my_insert_seq = "GGGAAATTTCCC" * 4  # Replace with your insert sequence
my_insert = Dseqrecord(my_insert_seq, name="MyInsert")

# Design primers
my_result = design_gibson_primers(
    vector=my_vector,
    insert=my_insert,
    insert_site=100,      # Change this to your desired position
    homology_length=40,
    target_tm=60.0
)

# Display results
print_primer_summary(my_result)
get_primers_for_ordering(my_result)